# Activity Log Routes

> routes for interacting with the activity log


In [1]:
# | default_exp routes.activity_log

In [2]:
# | exporti

import httpx

import domolibrary.client.get_data as gd
import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.DomoAuth as dmda

In [11]:
#| hide
import os
import pandas as pd
import pandas as pd
import datetime as dt

import domolibrary.utils.convert as convert


In [4]:
# | export
@gd.route_function
async def get_activity_log_object_types(
    auth: dmda.DomoAuth,
    parent_class: str = None,
    debug_num_stacks_to_drop = 1, 
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
) -> rgd.ResponseGetData:
    """retrieves a list of valid objectTypes that can be used to search the activity_log API"""

    url = f"https://{auth.domo_instance}.domo.com/api/audit/v1/user-audits/objectTypes"

    return await gd.get_data(
        url=url,
        method="GET",
        auth=auth,
        parent_class=parent_class,
        debug_api=debug_api,
        num_stacks_to_drop=debug_num_stacks_to_drop,
    )

#### sample implementation of get_activity_log_object_types


In [5]:
#| eval  :false

# import pandas as pd

try:
    token_auth = dmda.DomoTokenAuth(
        domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
        domo_instance="domo-community",
    )

    al_objecttype_res = await get_activity_log_object_types(auth=token_auth)

    print(pd.DataFrame(al_objecttype_res.response))

    # use to update `ActivityLog_ObjectType` enum
    # [ f"{objecttype.get('translation').upper().replace(' ', '_')} = '{objecttype.get('type')}'" for objecttype in  al_objecttype_res.response]

except Exception as e:
    print(e)

warning this token has not been validated by who_am_i, run get_auth_token first
                      type             translation
0             ACCESS_TOKEN            Access token
1                  ACCOUNT                 Account
2             ACTIVITY_LOG            Activity log
3                    ALERT                   Alert
4                      APP                     App
..                     ...                     ...
85            VIRTUAL_USER            Virtual user
86       WAREHOUSE_ACCOUNT       Warehouse account
87       WORKFLOW_INSTANCE       Workflow Instance
88          WORKFLOW_MODEL          Workflow Model
89  WORKFLOW_MODEL_VERSION  Workflow Model Version

[90 rows x 2 columns]


In [6]:
# | export
@gd.route_function
async def search_activity_log(
    auth: dmda.DomoAuth,
    start_time: int,  # epoch time in milliseconds
    end_time: int,  # epoch time in milliseconds
    maximum: int,
    object_type: str = None,
    debug_api: bool = False,
    debug_loop: bool = False,
    parent_class: str = None,
    debug_num_stacks_to_drop: int = 1,
    session: httpx.AsyncClient = None,
) -> rgd.ResponseGetData:
    """loops over activity log api to retrieve audit logs"""

    is_close_session = False

    if not session:
        session = httpx.AsyncClient()
        is_close_session = True

    url = f"https://{auth.domo_instance}.domo.com/api/audit/v1/user-audits"

    if object_type and object_type != "ACTIVITY_LOG":
        url = f"{url}/objectTypes/{object_type}"

    fixed_params = {"end": end_time, "start": start_time}

    offset_params = {
        "offset": "offset",
        "limit": "limit",
    }

    def arr_fn(res) -> list[dict]:
        return res.response

    res = await gd.looper(
        auth=auth,
        method="GET",
        url=url,
        arr_fn=arr_fn,
        fixed_params=fixed_params,
        offset_params=offset_params,
        session=session,
        maximum=maximum,
        debug_loop=debug_loop,
        debug_api=debug_api,
        debug_num_stacks_to_drop=debug_num_stacks_to_drop,
        parent_class=parent_class,
    )

    if is_close_session:
        await session.aclose()

    return res

In [12]:
#| eval : false

# import pandas as pd
# import datetime as dt

# import domolibrary.utils.convert as convert

end_datetime = dt.datetime.today()
start_datetime = end_datetime - dt.timedelta(days=100)

token_auth = dmda.DomoTokenAuth(
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
    domo_instance="domo-community",
)

page_res = await search_activity_log(
    object_type="PAGE",
    start_time=convert.convert_datetime_to_epoch_millisecond(start_datetime),
    end_time=convert.convert_datetime_to_epoch_millisecond(end_datetime),
    auth=token_auth,
    maximum=16,
    debug_loop=False,
    debug_api=False,
)

pd.DataFrame(page_res.response[0:5])

warning this token has not been validated by who_am_i, run get_auth_token first


,objectId,objectName
0,-100000,Overview
1,-100001,Alerts
2,-100002,Shared
3,-100003,Favorites
4,-100012,CEO


#### sample implementation of search_activity_log


In [13]:
#| eval : false

# import pandas as pd
# import datetime as dt
# import domolibrary.utils.convert as convert

end_datetime = dt.datetime.today()
start_datetime = end_datetime - dt.timedelta(days=100)

token_auth = dmda.DomoTokenAuth(
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
    domo_instance="domo-community",
)

activity_log_res = await search_activity_log(
    object_type="ACTIVITY_LOG",
    start_time=convert.convert_datetime_to_epoch_millisecond(start_datetime),
    end_time=convert.convert_datetime_to_epoch_millisecond(end_datetime),
    auth=token_auth,
    maximum=10,
    debug_loop=False,
    debug_api=False,
)

pd.DataFrame(activity_log_res.response[0:5])

warning this token has not been validated by who_am_i, run get_auth_token first


,userName,userId,userType,actorName,actorId,actionType,objectName,objectId,objectType,additionalComment,time,eventText,clientId,authenticationMethod
0,Jae Wilson1,1893952720,USER,,0,EXECUTED,tag_inactive_owners,743c1c6f-80d5-4b47-b02e-0ea28f6a5683,EXECUTOR_JOB,Jae Wilson1 executed Executor job tag_inactive...,1711218047106,Executed Executor Job,None,DEV_TOKEN
1,Jae Wilson1,1893952720,USER,,0,DELETED,DL_test,186953,ACCESS_TOKEN,Jae Wilson1 deleted access token DL_test. 8:26...,1711218000296,Access token deleted,None,DEV_TOKEN
2,Jae Wilson1,1893952720,USER,,0,CREATED,DL_test,186953,ACCESS_TOKEN,Jae Wilson1 created access token DL_test. 8:26...,1711217999954,Access token created,None,DEV_TOKEN
3,Jae Wilson1,1893952720,USER,,0,DELETED,DL_test,186952,ACCESS_TOKEN,Jae Wilson1 deleted access token DL_test. 8:26...,1711217964379,Access token deleted,None,DEV_TOKEN
4,Jae Wilson1,1893952720,USER,,0,CREATED,DL_test,186952,ACCESS_TOKEN,Jae Wilson1 created access token DL_test. 8:26...,1711217964112,Access token created,None,DEV_TOKEN


In [9]:
# | hide
import nbdev

nbdev.nbdev_export()
!nbqa black activity_log.ipynb

All done! ✨ 🍰 ✨
1 file left unchanged.
